In [1]:
%load_ext autoreload
%autoreload 2
import sys
sys.path.append("..")
from utils import *
from models import *

In [2]:
train_tape = "Z:/rppg/ccnu_datatape_128x32x32_train.h5"
valid_tape = "Z:/rppg/ccnu_datatape_128x32x32_valid.h5"

train =  load_datatape(train_tape, use_normalized_bvp=True)
valid = load_datatape(valid_tape, use_normalized_bvp=False)

#train, valid = to_tf(train).cache(f'{tmp}/physnet'), to_tf(valid).cache(f'{tmp}/physnet')
train, valid = to_tf(train), to_tf(valid)

In [3]:
phys_net = PhysNet()
phys_net.compile(optimizer=keras.optimizers.SGD(learning_rate=0.005), loss=np_loss)
phys_net.build(input_shape=(None, 128, 32, 32, 3))
print(f'Flops per frame:{get_flops(phys_net, input_sig=[tf.TensorSpec([1, 128, 32, 32, 3])])/128:.0f}')

Instructions for updating:
Use `tf.compat.v1.graph_util.tensor_shape_from_node_def_name`
Flops per frame:54260867


In [ ]:
phys_net.fit(train.shuffle(2000).batch(32), validation_data=valid.batch(32), epochs=3, callbacks=[keras.callbacks.ModelCheckpoint('../weights/PhysNet_CCNU.h5', monitor='val_loss', verbose=1, save_best_only=True, mode='min', save_weights_only=True)])
phys_net.load_weights('../weights/PhysNet_CCNU.h5')

In [4]:
phys_net.load_weights('../weights/PhysNet_CCNU.h5')

In [7]:
eval_on_dataset(test_set_CCNU, phys_net, 128, (32, 32), step=1, batch=32, save='../results/PhysNet_CCNU_CCNU.h5', sample=cv2.INTER_CUBIC)
get_metrics('../results/PhysNet_CCNU_CCNU.h5')

100%|██████████| 179/179 [07:36<00:00,  2.55s/it]


{'Sliding window': {'MAE': 1.118, 'RMSE': 4.113, 'R': 0.91636},
 'Whole video': {'MAE': 0.714, 'RMSE': 2.839, 'R': 0.95406}}

In [6]:
eval_on_dataset(test_set_CCNU_rPPG, phys_net, 128, (32, 32), step=1, batch=32, save='../results/PhysNet_CCNU_CCNU_rPPG.h5', sample=cv2.INTER_CUBIC)
get_metrics('../results/PhysNet_CCNU_CCNU_rPPG.h5')

100%|██████████| 54/54 [01:44<00:00,  1.94s/it]


{'Sliding window': {'MAE': 1.037, 'RMSE': 3.801, 'R': 0.92282},
 'Whole video': {'MAE': 0.684, 'RMSE': 1.927, 'R': 0.9791}}

In [6]:
eval_on_dataset(test_set_PURE, phys_net, 128, (32, 32), step=1, batch=32, save='../results/PhysNet_CCNU_PURE.h5', sample=cv2.INTER_CUBIC)
get_metrics('../results/PhysNet_CCNU_PURE.h5')

100%|██████████| 59/59 [00:28<00:00,  2.05it/s]


{'Sliding window': {'MAE': 0.745, 'RMSE': 4.264, 'R': 0.98284},
 'Whole video': {'MAE': 0.511, 'RMSE': 0.909, 'R': 0.99949}}

In [5]:
eval_on_dataset(test_set_UBFC_rPPG2, phys_net, 128, (32, 32), step=1, batch=32, save='../results/PhysNet_CCNU_UBFC.h5', sample=cv2.INTER_CUBIC)
get_metrics('../results/PhysNet_CCNU_UBFC.h5')

100%|██████████| 42/42 [00:48<00:00,  1.16s/it]


{'Sliding window': {'MAE': 1.74, 'RMSE': 4.351, 'R': 0.96863},
 'Whole video': {'MAE': 0.891, 'RMSE': 1.455, 'R': 0.99677}}